In [2]:
from numpy import *
from functools import reduce
import re
import os
import random
import jieba
import math
import operator

In [3]:
# 保留中文内容
# 爬虫下来的文档中会有冗余信息
def find_chinese(file):
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    chinese = re.sub(pattern, '', file)
    return chinese

In [4]:
# 准备数据：从文本中构建词向量
def loadDataSet2():
    # 0 --> 党政办公室
    # 1 --> 教务部
    # 2 --> 招生办公室
    # 3 --> 研究生院
    # 4 --> 科学技术部
    postingList = []
    classVec = []
    for file in os.listdir(f'data/'):
        text = open(f'data/{file}', 'r').read()
        text = find_chinese(text)
        
        text =  list(jieba.cut(text, cut_all=False))
        postingList.append(text)

        if file.startswith('党政办公室'):
            classVec.append(0)
        elif file.startswith('教务部'):
            classVec.append(1)
        elif file.startswith('招生办公室'):
            classVec.append(2)
        elif file.startswith('研究生院'):
            classVec.append(3)
        elif file.startswith('科学技术部'):
            classVec.append(4)
            
    return postingList, classVec

In [5]:
# 准备数据：从文本中构建词向量
def loadDataSet(goal):
    # 0 --> 党政办公室
    # 1 --> 教务部
    # 2 --> 招生办公室
    # 3 --> 研究生院
    # 4 --> 科学技术部
    postingList = []
    classVec = []
    for file in os.listdir(f'data1/{goal}'):
        text = open(f'/Users/alex_shen/SynologyDrive/PcBackup/深圳大学/课程/大三下/信息检索/实验/202102-信息检索-HW5/data1/{goal}/{file}', 'r').read()
        text = find_chinese(text)
        
        text =  list(jieba.cut(text, cut_all=False))
        postingList.append(text)

        if file.startswith('党政办公室'):
            classVec.append(0)
        elif file.startswith('教务部'):
            classVec.append(1)
        elif file.startswith('招生办公室'):
            classVec.append(2)
        elif file.startswith('研究生院'):
            classVec.append(3)
        elif file.startswith('科学技术部'):
            classVec.append(4)
            
    return postingList, classVec

In [6]:
# 构建一个包含所有文档词汇且不重复的词汇表
def createVocabList(dataSet):
    vocabSet = set([])  # 创建一个空集
    for document in dataSet:
        vocabSet = vocabSet | set(document)  # 创建两个集合的并集
    return list(vocabSet)

In [7]:
def concatenateTextOfAllDocsInClass(postingList, classVec, c):
    text_c = []
    for index, cat in enumerate(classVec):
        if cat == c:
            text_c.extend(postingList[index])
    return text_c

In [8]:
def trainNB(myVocabList, postingList, classVec):
    V = myVocabList
    N = len(postingList)
    condprob = {term: [0] * 5 for term in V}
    prior = [0] * 5
    for c in range(5):
        # 计算先验概率
        Nc = classVec.count(c)
        prior[c] = Nc / N
        # 拼接所有文档的词汇
        text_c = concatenateTextOfAllDocsInClass(postingList, classVec, c)
        # 计算条件概率
        for t in V:
            T_ct = text_c.count(t)
            condprob[t][c] = (T_ct + 1) / (len(text_c) + len(V))
    return V, prior, condprob
        

In [9]:
# 计算最大概率类别
def applyMultinomialNB(V, prior, condprob, text):
    p = []
    for c in range(5):
        p.append(math.log(prior[c], 2))
        for t in text:
            if t in V:
                p[c] += math.log(condprob[t][c], 2)
    return p.index(max(p))

In [10]:
if __name__ == '__main__':
    # # 随机取80%作为训练集，20%作为测试集
    # postingList, classVec = loadDataSet2()
    # c = list(zip(postingList, classVec))
    # random.shuffle(c)
    # postingList, classVec = zip(*c)
    # train_postingList, train_classVec = list(postingList[:int(len(postingList) * 0.8)]), list(classVec[:int(len(classVec) * 0.8)])
    # test_postingList, test_classVec = list(postingList[int(len(postingList) * 0.8):]), list(classVec[int(len(classVec) * 0.8):])
    # myVocabList = createVocabList(train_postingList)
    # V, prior, condprob = trainNB(myVocabList, train_postingList, train_classVec)
    
    # 每类文档取20个作为训练集，10个作为测试集
    train_postingList, train_classVec = loadDataSet('train')
    myVocabList = createVocabList(train_postingList) 
    
    test_postingList, test_classVec = loadDataSet('test')
    V, prior, condprob = trainNB(myVocabList, train_postingList, train_classVec)
    
    correct = [0] * 5
    for index, text in enumerate(test_postingList):
        res = applyMultinomialNB(V, prior, condprob, text)
        if res == test_classVec[index]:
            correct[test_classVec[index]] += 1
    
    # 0 --> 党政办公室
    # 1 --> 教务部
    # 2 --> 招生办公室
    # 3 --> 研究生院
    # 4 --> 科学技术部
    print('整体 正确率：', sum(correct) / len(test_postingList))
    print('党政办公室 正确率：', correct[0] / test_classVec.count(0))
    print('教务部 正确率：', correct[1] / test_classVec.count(1))
    print('招生办公室 正确率：', correct[2] / test_classVec.count(2))
    print('研究生院 正确率：', correct[3] / test_classVec.count(3))
    print('科学技术部 正确率：', correct[4] / test_classVec.count(4))
    

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/m0/dsy0kdw13rb2g65q1x0bqn_00000gn/T/jieba.cache
Loading model cost 0.344 seconds.
Prefix dict has been built successfully.


整体 正确率： 0.86
党政办公室 正确率： 0.9
教务部 正确率： 0.5
招生办公室 正确率： 1.0
研究生院 正确率： 1.0
科学技术部 正确率： 0.9
